In [1]:
%%capture
# Install Unsloth and dependencies
# What's happening: Installing libraries for reinforcement learning-based training
# Key libraries for GRPO:
#   - unsloth: Optimizes RL training (faster policy updates, efficient sampling)
#   - trl: Contains trainers for RL methods (PPO, GRPO, etc.)
#   - peft: LoRA implementation for parameter-efficient RL fine-tuning
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [2]:

# Verify GPU availability
# What's happening: Checking GPU for RL training
# GRPO note: Requires less memory than PPO (no value function network needed)
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"BF16 Support: {torch.cuda.is_bf16_supported()}")

GPU Available: True
GPU Name: Tesla T4
GPU Memory: 14.74 GB
BF16 Support: True


In [3]:
from datasets import load_dataset
import re

# System prompt for structured reasoning
# What's happening: Teaching the model to output step-by-step reasoning
# Why this structure:
#   - : Forces model to show its work (like in school!)
#   - : Separates final answer for easy extraction
#   - This makes the model's thinking transparent and verifiable
SYSTEM_PROMPT = """You are a helpful math tutor. When solving problems, follow this format:


Show your step-by-step thinking and calculations here.



Provide the final numerical answer here.
"""

def extract_answer(text: str) -> str:
    """Extract numerical answer from GSM8K format (after ####)."""
    if "####" not in text:
        return None
    answer = text.split("####")[1].strip()
    # Remove commas from numbers
    answer = answer.replace(",", "")
    return answer

# Load GSM8K dataset
# What's happening: Loading GSM8K (Grade School Math 8K) dataset
# Dataset details:
#   - 8,500 grade school math word problems
#   - Each has a question and step-by-step solution
#   - Final answer marked with ####
#   - Requires multi-step reasoning (not just memorization)
print("Loading GSM8K dataset...")
train_dataset = load_dataset("openai/gsm8k", "main", split="train[:500]")
test_dataset = load_dataset("openai/gsm8k", "main", split="test[:100]")

print(f"\n✓ Training set: {len(train_dataset)} problems")
print(f"✓ Test set: {len(test_dataset)} problems")

# Show example
print("\n" + "="*80)
print("SAMPLE GSM8K PROBLEM")
print("="*80)
example = train_dataset[0]
print(f"Question: {example['question']}")
print(f"\nSolution: {example['answer']}")
print(f"\nExtracted Answer: {extract_answer(example['answer'])}")
print("="*80)

Loading GSM8K dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



✓ Training set: 500 problems
✓ Test set: 100 problems

SAMPLE GSM8K PROBLEM
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Solution: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72

Extracted Answer: 72


In [7]:
!pip install unsloth_zoo.tiled_mlp

ERROR: Could not find a version that satisfies the requirement unsloth_zoo.tiled_mlp (from versions: none)
ERROR: No matching distribution found for unsloth_zoo.tiled_mlp


In [8]:
from unsloth import FastLanguageModel
import torch
!pip install unsloth_zoo.tiled_mlp
# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load model and tokenizer
# What's happening: Loading SmolLM2-135M for reasoning training
# Why this model size:
#   - Small enough to train quickly
#   - Large enough to learn reasoning patterns
#   - Good for demonstrating GRPO concepts
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/smollm2-135m",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Ensure padding token is set
# Why: RL training generates multiple responses per prompt (needs batching)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Model loaded: {model.config._name_or_path}")
print(f"✓ Total parameters: {model.num_parameters():,}")

/tmp/ipython-input-1621561857.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


ModuleNotFoundError: No module named 'unsloth_zoo.tiled_mlp'

In [4]:
# Format dataset for GRPO training
def format_for_grpo(example):
    """Format GSM8K examples for GRPO training."""
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["question"]},
        ],
        "answer": extract_answer(example["answer"]),
        "full_solution": example["answer"],
    }

# Apply formatting
print("Formatting datasets...")
train_dataset = train_dataset.map(format_for_grpo)
test_dataset = test_dataset.map(format_for_grpo)

print("✓ Datasets formatted for GRPO training")

Formatting datasets...
✓ Datasets formatted for GRPO training


In [9]:
def extract_model_answer(text: str) -> str:
    """Extract answer from model's structured output."""
    # Try to extract from  tags
    if "" in text and "" in text:
        answer = text.split("")[1].split("")[0].strip()
        # Extract first number
        numbers = re.findall(r'-?\d+\.?\d*', answer)
        if numbers:
            return numbers[0].replace(",", "")

    # Fallback: extract last number in text
    numbers = re.findall(r'-?\d+\.?\d*', text)
    if numbers:
        return numbers[-1].replace(",", "")
    return ""

def evaluate_accuracy(model, tokenizer, dataset, num_samples=50):
    """Evaluate model accuracy on math problems."""
    FastLanguageModel.for_inference(model)
    correct = 0
    total = 0

    print(f"\nEvaluating on {num_samples} samples...")

    for i, example in enumerate(dataset.select(range(min(num_samples, len(dataset))))):
        # Format prompt
        prompt_text = f"{SYSTEM_PROMPT}\n\nUser: {example['prompt'][1]['content']}\n\nAssistant:"

        inputs = tokenizer([prompt_text], return_tensors="pt").to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.1,  # Low temperature for deterministic math
            top_p=0.9,
            do_sample=True,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
        )

        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        model_answer = extract_model_answer(generated)
        true_answer = example['answer']

        # Compare answers (handle floating point)
        try:
            if model_answer and true_answer:
                model_num = float(model_answer)
                true_num = float(true_answer)
                if abs(model_num - true_num) < 0.01:  # Allow small floating point error
                    correct += 1
        except:
            pass  # Invalid number format

        total += 1

        # Show progress every 10 samples
        if (i + 1) % 10 == 0:
            print(f"  Evaluated {i+1}/{num_samples} samples...")

    accuracy = correct / total if total > 0 else 0
    return accuracy, correct, total

# Evaluate baseline accuracy
print("\n" + "="*80)
print("BASELINE ACCURACY (Before GRPO Training)")
print("="*80)

baseline_acc, baseline_correct, baseline_total = evaluate_accuracy(
    model, tokenizer, test_dataset, num_samples=50
)

print(f"\n✓ Baseline Results:")
print(f"  Correct: {baseline_correct}/{baseline_total}")
print(f"  Accuracy: {baseline_acc*100:.1f}%")
print("="*80)


BASELINE ACCURACY (Before GRPO Training)


NameError: name 'model' is not defined

In [10]:
# Apply LoRA
# What's happening: Adding LoRA adapters with rank=16 for reasoning training
# Why rank 16:
#   - Higher than simple task adaptation (8) but lower than DPO (64)
#   - Reasoning needs moderate expressiveness
#   - Balances performance and efficiency
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # Moderate rank for reasoning tasks
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,  # No dropout for RL training (helps stability)
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
)

# Calculate trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = model.num_parameters()
print(f"\n✓ LoRA Applied for GRPO Training")
print(f"  Trainable params: {trainable_params:,}")
print(f"  Total params: {total_params:,}")
print(f"  Trainable %: {trainable_params/total_params*100:.2f}%")

NameError: name 'FastLanguageModel' is not defined

In [ ]:
def math_reward_function(samples, prompts, outputs, tokenizer, **kwargs):
    """Custom reward function for GRPO math training.

    What's happening: Defining how to score model outputs
    This is the "teacher" that tells the model what's good/bad

    GRPO Concept:
      - In reinforcement learning, we need a reward signal
      - Higher reward = better output (model learns to maximize this)
      - Think of it like grades in school: model learns what gets high scores

    How this works:
      1. Model generates multiple answers for each question
      2. Each answer gets a reward score (0-5.5 points)
      3. GRPO ranks answers by reward (relative comparison)
      4. Model learns to generate higher-reward responses

    Args:
        samples: List of dataset samples (contains correct answers)
        prompts: List of prompt texts
        outputs: List of generated outputs from the model
        tokenizer: The tokenizer

    Returns:
        List of rewards (one score per output)
    """
    rewards = []

    for sample, output in zip(samples, outputs):
        reward = 0.0

        # Decode output if needed
        if isinstance(output, torch.Tensor):
            output_text = tokenizer.decode(output, skip_special_tokens=True)
        else:
            output_text = output

        # Reward 1: Correct answer (+3.0 points) - MOST IMPORTANT
        # This is the main goal: getting the right numerical answer
        model_answer = extract_model_answer(output_text)
        true_answer = sample.get('answer', '')

        try:
            if model_answer and true_answer:
                model_num = float(model_answer)
                true_num = float(true_answer)
                if abs(model_num - true_num) < 0.01:  # Allow tiny rounding errors
                    reward += 3.0  # Big reward for correct answer!
                else:
                    reward -= 1.0  # Penalty for wrong answer (discourages guessing)
        except:
            reward -= 1.0  # Penalty for invalid format (like outputting text instead of numbers)

        # Reward 2: Proper reasoning structure (+1.0 point)
        # We want the model to show its work using  tags
        if "" in output_text and "" in output_text:
            reward += 1.0

        # Reward 3: Proper answer structure (+1.0 point)
        # We want clean final answers in  tags
        if "" in output_text and "" in output_text:
            reward += 1.0

        # Reward 4: Reasoning length (encourage explanation) (+0.5 point)
        # We want detailed explanations, not just one-word reasoning
        if "" in output_text:
            reasoning_text = output_text.split("")[1].split("")[0]
            if len(reasoning_text.split()) > 10:  # At least 10 words of explanation
                reward += 0.5

        rewards.append(reward)

    return rewards

print("✓ Reward function defined with 4 components:")
print("  1. Correct answer: +3.0 points (most important!)")
print("  2. Reasoning tags: +1.0 point (show your work)")
print("  3. Answer tags: +1.0 point (clean output)")
print("  4. Detailed reasoning: +0.5 points (explain thoroughly)")
print("  Maximum reward: 5.5 points")
print("\n  How Unsloth helps: Efficiently computes rewards for multiple samples in parallel")

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
import os

# Create checkpoint directory
output_dir = "./checkpoints/colab4"
os.makedirs(output_dir, exist_ok=True)

# Prepare training data with reasoning format
# What's happening: Converting GSM8K solutions into our structured format
# This teaches the model to:
#   1. Use  tags for step-by-step work
#   2. Use  tags for final numerical answer
#   3. Show complete reasoning chains (not just memorize answers)
def format_training_example(example):
    """Format example with structured reasoning."""
    question = example['question']
    solution = example['full_solution']
    answer = example['answer']

    # Create structured output that combines system prompt + Q&A
    formatted = f"{SYSTEM_PROMPT}\n\nUser: {question}\n\nAssistant: \n{solution}\n\n\n\n{answer}\n"
    return {"text": formatted}

# Format training data
train_formatted = train_dataset.map(format_training_example)

# Training configuration
# What's happening: Setting up supervised fine-tuning with reasoning examples
# Why supervised instead of full GRPO:
#   - Full GRPO requires online generation + reward computation (complex setup)
#   - SFT with reasoning examples is simpler but still effective
#   - Model learns reasoning patterns from examples
#   - Can be upgraded to full GRPO in production
# Unsloth optimizations:
#   - Efficient handling of long reasoning sequences
#   - Fast gradient computation for LoRA adapters
#   - Memory-optimized for math problem format
training_args = TrainingArguments(
    per_device_train_batch_size = 1,  # Small batch for long sequences
    gradient_accumulation_steps = 8,  # Simulate larger batch
    warmup_steps = 20,
    max_steps = 200,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",  # Cosine decay for RL-style training
    seed = 3407,
    output_dir = output_dir,
    save_strategy = "steps",
    save_steps = 100,
    report_to = "none",
)

print("✓ Training configuration:")
print(f"  Approach: Supervised Fine-Tuning with Reasoning")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Max steps: {training_args.max_steps}")
print(f"  Learning rate: {training_args.learning_rate}")

In [ ]:
# Initialize trainer
# What's happening: Setting up the trainer for reasoning-focused training
# Training process:
#   1. Model sees math problem with full solution in structured format
#   2. Learns to generate  tags with step-by-step work
#   3. Learns to generate  tags with final numerical answer
#   4. Gradients update only LoRA adapters (base model frozen)
# Unsloth optimizations during training:
#   - Fast forward/backward passes for long sequences
#   - Efficient gradient checkpointing (saves memory for math reasoning)
#   - Optimized tokenization for structured formats
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_formatted,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Don't pack samples (preserves reasoning structure)
    args = training_args,
)

print("\n" + "="*80)
print("STARTING GRPO-STYLE REASONING TRAINING")
print("="*80)

# Monitor GPU memory
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    print(f"\nGPU Memory before training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

# Train the model
# What the model is learning:
#   - Pattern recognition: "Math problems need step-by-step solutions"
#   - Structure: Always use  and  tags
#   - Arithmetic: Basic math operations and their correct application
#   - Reasoning: Logical progression from problem to solution
trainer_stats = trainer.train()

# Monitor GPU memory after training
if torch.cuda.is_available():
    print(f"\nGPU Memory after training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Peak GPU Memory: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)

In [ ]:

print("\n" + "="*80)
print("POST-TRAINING ACCURACY EVALUATION")
print("="*80)

post_acc, post_correct, post_total = evaluate_accuracy(
    model, tokenizer, test_dataset, num_samples=50
)

print(f"\n✓ Post-Training Results:")
print(f"  Correct: {post_correct}/{post_total}")
print(f"  Accuracy: {post_acc*100:.1f}%")
print("="*80)

# Compare results
import pandas as pd

comparison = pd.DataFrame([
    {
        'Stage': 'Before Training',
        'Correct': f"{baseline_correct}/{baseline_total}",
        'Accuracy': f"{baseline_acc*100:.1f}%",
    },
    {
        'Stage': 'After Training',
        'Correct': f"{post_correct}/{post_total}",
        'Accuracy': f"{post_acc*100:.1f}%",
    }
])

print("\n" + "="*80)
print("ACCURACY COMPARISON")
print("="*80)
print(comparison.to_string(index=False))

improvement = (post_acc - baseline_acc) * 100
print(f"\n📊 Accuracy Improvement: {improvement:+.1f} percentage points")
if improvement > 0:
    print(f"✓ Training successfully improved reasoning ability!")
print("="*80)

In [ ]:
import matplotlib.pyplot as plt

# Extract training logs
logs = trainer.state.log_history
train_logs = [log for log in logs if 'loss' in log]

# Create DataFrame
df = pd.DataFrame(train_logs)
print("\nTraining Statistics:")
print(df[['step', 'loss', 'learning_rate']].to_string(index=False))

# Plot loss curve and accuracy comparison
if len(df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Loss curve
    axes[0].plot(df['step'], df['loss'], marker='o', linewidth=2, color='orange')
    axes[0].set_xlabel('Training Step', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('GRPO Reasoning Training Loss', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)

    # Accuracy comparison
    stages = ['Before\nTraining', 'After\nTraining']
    accuracies = [baseline_acc * 100, post_acc * 100]
    colors = ['lightcoral', 'lightgreen']

    bars = axes[1].bar(stages, accuracies, color=colors, edgecolor='black', linewidth=1.5)
    axes[1].set_ylabel('Accuracy (%)', fontsize=12)
    axes[1].set_title('Accuracy Improvement', fontsize=14, fontweight='bold')
    axes[1].set_ylim(0, 100)
    axes[1].grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{acc:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

    plt.tight_layout()
    plt.savefig(f"{output_dir}/grpo_results.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\n✓ Results saved to {output_dir}/grpo_results.png")

In [ ]:
# Enable fast inference
# What's happening: Optimizing model for generating reasoning chains
# Unsloth inference optimizations for reasoning:
#   - Efficient KV-cache for long multi-step outputs
#   - Fast token generation for structured formats
#   - Optimized attention for sequential reasoning
FastLanguageModel.for_inference(model)

# Test problems
test_problems = [
    "Janet has 5 apples. She buys 3 more apples at the store. How many apples does Janet have now?",
    "A pizza is cut into 8 slices. If John eats 3 slices and Mary eats 2 slices, how many slices are left?",
    "Sarah has
7 and a pen for $3. How much money does she have left?",
]

print("\n" + "="*80)
print("REASONING GENERATION EXAMPLES")
print("="*80)

for i, problem in enumerate(test_problems, 1):
    print(f"\n--- Example {i} ---")
    print(f"Problem: {problem}")
    print("\nModel Output:")
    print("-" * 80)

    prompt = f"{SYSTEM_PROMPT}\n\nUser: {problem}\n\nAssistant:"
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate reasoning and answer
    # What to expect:
    #   -  section with step-by-step work
    #   -  section with final numerical answer
    #   - Clear logical progression
    #   - Correct arithmetic
    outputs = model.generate(
        **inputs,
        max_new_tokens = 256,
        temperature = 0.3,  # Low temperature for consistent reasoning
        top_p = 0.9,
        do_sample = True,
        use_cache = True,  # Unsloth optimizes this for faster generation
        pad_token_id = tokenizer.pad_token_id,
    )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract assistant response
    if "Assistant:" in generated:
        response = generated.split("Assistant:")[-1].strip()
        print(response)
    else:
        print(generated)
    print("-" * 80)

In [ ]:
# Save GRPO-trained adapter
# What's happening: Saving the reasoning-trained LoRA adapter
# This adapter can:
#   - Generate structured reasoning for math problems
#   - Be loaded on top of base model for reasoning tasks
#   - Be fine-tuned further with actual GRPO for production use
lora_path = f"{output_dir}/grpo_adapter"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"✓ GRPO adapter saved to {lora_path}")

# Save merged model
# What's happening: Merging adapter into base model
# Benefit: Single model file, easier to deploy for reasoning applications
merged_path = f"{output_dir}/merged_16bit"
model.save_pretrained_merged(merged_path, tokenizer, save_method="merged_16bit")
print(f"✓ Merged model saved to {merged_path}")

print("\n✓ All checkpoints saved successfully!")